In [1]:
import pandas as pd
import ast
import json
import re
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import warnings
from scipy.stats import pearsonr
warnings.filterwarnings('ignore')
from itertools import combinations
import random
from seqeval.metrics import f1_score,classification_report
from sklearn.metrics import f1_score as bi_f1_score
from sklearn.metrics import classification_report as bi_classification_report
import traceback
import html #for unescape & < >
from scipy import stats
import matplotlib.pyplot as plt

import emoji
import os

In [2]:
aa=pd.read_json('all_processed.json')
pd.set_option('precision', 10)

def workeridx_filter_binary_entity_f1(entity_label_binary,threshold):    
    ass_number = len(entity_label_binary)
    t_f1 = np.empty((ass_number, ass_number)) * np.nan
    for i in range(ass_number):
        for j in range(ass_number):
            t_f1[i,j] = bi_f1_score(entity_label_binary[i], entity_label_binary[j],pos_label=1,average='binary')
                
            if i==j:
                t_f1[i,j] = np.nan #assign diagonal value as nan for computing conveniently 
               

    my_score_f1 = np.nanmean(t_f1,axis=0)
    max_f1 = np.nanmax(my_score_f1)
    bad_entity_widx_f1 = np.where(my_score_f1/max_f1<threshold)[0]
    best_annotation_idx = np.nanargmax(my_score_f1)
    return bad_entity_widx_f1, best_annotation_idx
def comput_dist_categories(entity_list):
    entity_labels=[['B-food'],['B-loc'],['B-symptom'],['B-other']]
    c_list=[]
    for category in entity_labels:
        count=0
        for j in category:
            count += entity_list.count(j)
        c_list.append(count)
    return [np.array(c_list),sum(c_list)]

In [4]:
aa['entity_label_count']=aa['entity_label'].apply(lambda y:[comput_dist_categories(x) for x in y])

aa['entity_label_binary']=aa['entity_label'].apply(lambda y:[[0 if xi=='O' else 1 for xi in x] for x in y])




In [6]:
             

threshold=0.6
idx_f1=aa['entity_label_binary'].apply(lambda x: workeridx_filter_binary_entity_f1(x,threshold))

aa['worker_noentity_id']=aa['workerid']


aa['worker_filter_entity_f1']=aa['workerid']
aa['worker_filter_entity_f1_id']=aa['workerid']
aa['best_annotation_idx']=aa['workerid']
for i in range(aa.shape[0]):
    idx_s=np.array(list(set(idx_f1.iloc[i][0])))
    aa['best_annotation_idx'].iloc[i] = idx_f1.iloc[i][1]              
    if len(idx_s)>0:
        aa['worker_filter_entity_f1'].iloc[i] = np.array(aa['workerid'].iloc[i])[idx_s].tolist()
        aa['worker_filter_entity_f1_id'].iloc[i] = list(idx_s)

    else:
        aa['worker_filter_entity_f1'].iloc[i]=[]
        aa['worker_filter_entity_f1_id'].iloc[i] = []


aa['assignmentID_filter']=aa['assignmentID']
for i in range(aa.shape[0]):
#             aa['assignmentID_filter'].iloc[i]=np.array(aa['assignmentID'].iloc[i])[list(set(aa['worker_filter_entity_f1_id'].iloc[i]).intersection(set(aa['worker_filter_binary_entity_auc+cor_id'].iloc[i])))]
    aa['assignmentID_filter'].iloc[i]=np.array(aa['assignmentID'].iloc[i])[list(aa['worker_filter_entity_f1_id'].iloc[i])]
#             aa['assignmentID_filter'].iloc[i]=np.array(aa['assignmentID'].iloc[i])[aa['worker_filter_entity_f1_id'].iloc[i]]

aa['keep_idx']=aa['assignmentID']
for i in range(aa.shape[0]):
    aa['keep_idx'].iloc[i] = list(set(range(len(aa['assignmentID'].iloc[i])))-set(aa['worker_filter_entity_f1_id'].iloc[i]))

aa['keep_number']=aa['workerid'].apply(len)-aa['assignmentID_filter'].apply(len)
        
        




In [8]:
## creat files for kept ones in  initial batch and main batches

data_3000=pd.read_csv('main_batches.csv')
data_200=pd.read_csv('initial_batch.csv')

keep_assignmentID=list(set(sum(aa.assignmentID.tolist(),[]))-set(sum(aa.assignmentID_filter.apply(lambda x: list(x)).tolist(),[])))

keep_df=pd.DataFrame()
keep_df['AssignmentID']=keep_assignmentID


new_keep_3000=data_3000.merge(keep_df,left_on='AssignmentId',right_on='AssignmentID',how='inner')
new_keep_200=data_200.merge(keep_df,left_on='AssignmentId',right_on='AssignmentID',how='inner')



new_keep_3000.to_json('filtered_main_batches.json')
new_keep_200.to_json('filtered_initial_batches.json')
